In [2]:
import numpy as np
import torch
from core.model import Model
import skdim
import yaml
import copy

In [14]:
def estimate_dim(path, coords, d, c, N=1000, scale=1e-5):

    log_dir = "lightning_logs/" + path
    config_path = log_dir + "config.yaml"
    ckpt_path = log_dir + "/checkpoints/last.ckpt"
    
    with open(config_path, "r") as file:
        config = yaml.safe_load(file)

    #Load model
    model = Model.load_from_checkpoint(ckpt_path, input_shape=((1,1),(1,1,1,d)), output_shape=(1,1,c), **config["model"])
    state = copy.deepcopy(model.state_dict())

    N = 1000
    samples = np.zeros((N, coords[1].shape[2], c))

    scale = 1e-5

    for i in range(N):
        #Perturb
        for param in model.parameters():
            if param.requires_grad:
                with torch.no_grad():
                    param.data.add_(scale*torch.randn(param.shape))
    
        #Evaluate
        with torch.no_grad():
            output = model(coords)
    
        #Append sample
        samples[i,:] = output.detach().numpy() #channel 1
    
        #Reset state
        model.load_state_dict(state)

    dim = 0.
    
    for i in range(c):
        lpca = skdim.id.lPCA().fit_pw(samples[:,:,i], n_neighbors=100, n_jobs=1)

        print(np.mean(lpca.dimension_pw_), lpca.dimension_)

        dim += lpca.dimension_

    return dim/c

# Ignition Dataset

In [12]:
points = torch.from_numpy(np.load("data/ignition/points.npy").astype(np.float32)).expand(1, 1, -1, -1)
t = torch.tensor([[0.]])

coords = (t, points)

path = "ignition/hnet_online_manifold/version_0/"

In [15]:
estimate_dim(path, coords, 2, 4)

Exact parameter count: 31652
10.26 8
8.18 9
8.717 10
7.921 9
